In [ ]:
import gensim
import codecs
import json

from math import ceil

from gensim.models.keyedvectors import Word2VecKeyedVectors
from gensim.models import KeyedVectors
from gensim.models import FastText
from gensim.scripts.glove2word2vec import glove2word2vec

import collections
import copy
import itertools
import random
import sklearn

import re
import tqdm

import warnings
warnings.filterwarnings("ignore")

import importlib
import pickle
from collections import defaultdict, Counter
from typing import List, Dict

import os 
from os.path import join

import torch
from torch import utils
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np

import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

import ref 

EPOCHSTEPS = 1000000
STOPWORDS = set(stopwords.words('english'))

In [ ]:
#get embeddings 

def load_word_vectors(fname):
    model = KeyedVectors.load_word2vec_format(fname, limit=200000, binary=False)
    vecs = model.vectors
    words = list(model.vocab.keys())
    return model, vecs, words

word2vec, vecs, words = load_word_vectors("crawl-300d-2M.vec")

In [ ]:
def sent_proc(txt, stopwords=[], lem=None):
    words = txt.split(" ")
    words = {re.sub(r'\W+', '', w).lower() for w in words \
             if re.sub(r'\W+', '', w).lower() not in stopwords}
    if lem is not None: 
        words = {lem.lemmatize(w) for w in words}
    return words

# Data Cleaning

In [ ]:
class ToxicityDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data, transform=None):
        """
        Args:
            data (pd dataframe): The pd dataframe with (uid, tox_label, text)
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.dataset = data
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset.loc[idx, 'comment_text']
        if self.transform:
            sample = self.transform(sample)
        
        target = self.dataset.loc[idx, 'toxicity']
        
        return {'x':sample, 'y':np.expand_dims(target.values, axis=1)} #Fix this without hack
    
class GetEmbedding(object):
    """Given a sentence of text, generate sentence embedding

    Args:
        model: word embedding model, dictionary of wrods -> embeds
    """

    def __init__(self, model, stopwords=[]):
        self.model = model
        self.stopwords = stopwords
        self.unknown_embed = np.zeros(300)  #NOTE - this may not be same for all WEs 
        
    def __call__(self, sample):
        ''':param sample: pd.Series'''
        if type(sample) == str:
            words = sample.split(" ")
            words = [w for w in words if w.lower() not in self.stopwords]
            sent_embedding = np.sum([self.model[w] if w in self.model else self.unknown_embed for w in words], axis = 0)    
        elif type(sample) == pd.Series:
            sent_embedding = np.zeros((len(sample), 300))
            for i, txt in enumerate(sample):
                words = txt.split(" ")
                words = [w for w in words if w.lower() not in self.stopwords]
                sent_embedding[i, :] = np.sum([self.model[w] if w in self.model else self.unknown_embed for w in words], axis = 0)   
        
        return sent_embedding
    
class GetBOW(object):
    """Given a sentence of text, generate BOW rep

    Args:
        vocab: dictionary, word-->index in array (assume contigious) """

    def __init__(self, vocab, lem=None, stopwords=[]):
        self.vocab = vocab
        self.stopwords = stopwords
        self.lem = lem
        
    def __call__(self, sample):
        ''':param sample: str or pd.Series'''
        def get_rep(txt):
            rep = np.zeros(len(self.vocab))
            words = sent_proc(txt, stopwords=self.stopwords, lem=self.lem)
            for w in words:
                try:
                    rep[self.vocab[w]] = 1
                except KeyError:
                    continue
            return rep
        
        if type(sample) == str:
            bow_embed = get_rep(sample)
        elif type(sample) == pd.Series:
            bow_embed = np.zeros((len(sample), len(self.vocab)))
            for i, txt in enumerate(sample):
                bow_embed[i, :] = get_rep(txt)
                
        return bow_embed

#Data Functions
def generate_dataset(d, elg, t=None):
    full = d[['id', 'toxicity', 'comment_text']]
    full = full[elg]
    
    #convert to pytorch formatting
    full = ToxicityDataset(full, transform=t)
    return full

def generate_dataloader(d, elg, nbatch=1, t=None):
    full = generate_dataset(d, elg, t=t)
    full = DataLoader(full, ceil(len(full)/nbatch), shuffle=False)
    return full

In [ ]:
full_data = pd.read_csv('all_data.csv')
print(full_data.columns)

# Baseline

In [ ]:
seeds = [10000, 8079, 501]
splits = [np.array([[0.1, 0.9], [0.2, 0.8], [0.9, 0.1]]), \
          np.array([[0.1, 0.9], [0.3, 0.7], [0.9, 0.1]]), \
          np.array([[0.05, 0.95], [0.35, 0.65], [0.9, 0.1]])]
label_noise = [0, 0.25]

In [ ]:
thresh = 0.2
#Data Cleaning 
full_data['LGTBQ'] = full_data[['homosexual_gay_or_lesbian', 'bisexual', 'other_sexual_orientation']].max(axis=1)

full_partition = full_data[(full_data['LGTBQ'] > 0)]
toxic, non_toxic = full_partition[full_data['toxicity'] >= thresh].sample(frac=1).reset_index(drop=True), \
                        full_partition[full_data['toxicity'] < thresh].sample(frac=1).reset_index(drop=True)
toxic['toxicity'], non_toxic['toxicity'] = toxic['toxicity'].apply((lambda x: 1 if x > thresh else 0)), \
                    non_toxic['toxicity'].apply((lambda x: 1 if x > thresh else 0))

totals = {'nt':len(non_toxic), 't':len(toxic)}
env_splits = np.array([[.1, 0.9], [0.2, 0.8], [0.9, 0.1]]) 
weights = {'nt':env_splits.mean(axis=0)[0], 't':env_splits.mean(axis=0)[1]}

#Adjust so that desired env splits possible

if float(totals['t']/(totals['t'] + totals['nt'])) >= weights['t']:  #see who has the bigger proportion 
    ns = int(totals['nt']/weights['nt'] - totals['nt'])   #     int((len(full_partition) - weights['nt']*totals['nt'])/weights['t'])
    toxic = toxic.sample(n=ns)
else:
    ns = int(totals['t']/weights['t'] - totals['t']) 
    non_toxic = non_toxic.sample(n=ns)
    
#partition env splits
nenvs = env_splits.shape[0]
e_props = env_splits/env_splits.sum(axis=0) #proprotion of vector in each env

env_partitions = []  #Note - last env is the test env
for i in range(nenvs):  #Note - tehre might be an error here that excludes  single sample from diff envs 
    #Get both componenets of envs 
    past_ind = int(np.array(e_props[:i, 0]).sum() * len(non_toxic))    
    pres_ind = int(np.array(e_props[:(i+1), 0]).sum() * len(non_toxic))
    nt = non_toxic.iloc[past_ind:pres_ind]

    past_ind = int(np.array(e_props[:i, 1]).sum() * len(toxic))    
    pres_ind = int(np.array(e_props[:(i+1), 1]).sum() * len(toxic))
    t = toxic.iloc[past_ind:pres_ind]
    
    #Make full env 
    env_partitions.append(pd.concat([nt, t], ignore_index=True).sample(frac=1))
    
#Baseline Logistic Regression
t = GetEmbedding(word2vec, stopwords=STOPWORDS)
train_partition = ToxicityDataset(pd.concat([e for e in env_partitions[:-1]], \
                                            ignore_index=True)[['id', 'toxicity', 'comment_text']], transform=t)[:]  
test_partition = ToxicityDataset(env_partitions[-1][['id', 'toxicity', 'comment_text']], transform=t)[:]

print(train_partition['x'].shape, test_partition['x'].shape)
model = LogisticRegression(fit_intercept = True, penalty = 'l2').fit(train_partition['x'], train_partition['y'])
print('train score: {}'.format(model.score(train_partition['x'], train_partition['y'])))
print('test score: {}'.format(model.score(test_partition['x'], test_partition['y'])))
pickle.dump(model, open('results/baseline.pkl', 'wb'))

#IRM Logistic Regression 
t = GetEmbedding(word2vec, stopwords=STOPWORDS)
train_envs = [ToxicityDataset(e[['id', 'toxicity', 'comment_text']], transform=t)[:] for e in env_partitions[:-1]]
test_partition = ToxicityDataset(env_partitions[-1][['id', 'toxicity', 'comment_text']], transform=t)[:]

print(train_envs[0]['x'].shape, test_partition['x'].shape)

args = {'lr': 0.0001, \
         'n_iterations':70000, \
         'penalty_anneal_iters':1, \
         'l2_reg':1.0, \
         'pen_wgt':1000, \
         'hid_layers':1, \
         'verbose':False}
base = ref.LinearInvariantRiskMinimization('cls')
model, errors, penalties, losses = base.train(train_envs, 1000, args)

train_logits = base.predict(np.concatenate([train_envs[i]['x'] for i in range(len(train_envs))]), model)
train_labels = np.concatenate([train_envs[i]['y'] for i in range(len(train_envs))])
test_logits = base.predict(test_partition['x'], model)
test_labels = test_partition['y']

train_acc = ref.compute_loss(np.expand_dims(train_logits, axis=1), train_labels, ltype='ACC')
test_acc = ref.compute_loss(np.expand_dims(test_logits, axis=1), test_labels, ltype='ACC')
print('train score: {}'.format(train_acc))
print('test score: {}'.format(test_acc))
pickle.dump(model, open('results/irm.pkl', 'wb'))


In [ ]:
#BUFFER DUDE

In [ ]:
#partition env splits
nenvs = env_splits.shape[0]
e_props = env_splits/env_splits.sum(axis=0) #proprotion of vector in each env

env_partitions = []  #Note - last env is the test env
for i in range(nenvs):  #Note - tehre might be an error here that excludes  single sample from diff envs 
    #Get both componenets of envs 
    past_ind = int(np.array(e_props[:i, 0]).sum() * len(non_toxic))    
    pres_ind = int(np.array(e_props[:(i+1), 0]).sum() * len(non_toxic))
    nt = non_toxic.iloc[past_ind:pres_ind]

    past_ind = int(np.array(e_props[:i, 1]).sum() * len(toxic))    
    pres_ind = int(np.array(e_props[:(i+1), 1]).sum() * len(toxic))
    t = toxic.iloc[past_ind:pres_ind]
    
    #Make full env 
    env_partitions.append(pd.concat([nt, t], ignore_index=True).sample(frac=1))


## Baseline Logistic Regression

In [ ]:
t = GetEmbedding(word2vec, stopwords=STOPWORDS)
train_partition = ToxicityDataset(pd.concat([e for e in env_partitions[:-1]], \
                                            ignore_index=True)[['id', 'toxicity', 'comment_text']], transform=t)[:]  
test_partition = ToxicityDataset(env_partitions[-1][['id', 'toxicity', 'comment_text']], transform=t)[:]

print(train_partition['x'].shape, test_partition['x'].shape)

In [ ]:
model = LogisticRegression(fit_intercept = True, penalty = 'l2').fit(train_partition['x'], train_partition['y'])
print('train score: {}'.format(model.score(train_partition['x'], train_partition['y'])))
print('test score: {}'.format(model.score(test_partition['x'], test_partition['y'])))

## IRM Logistic Regression

In [ ]:
import ref 
t = GetEmbedding(word2vec, stopwords=STOPWORDS)
train_envs = [ToxicityDataset(e[['id', 'toxicity', 'comment_text']], transform=t)[:] for e in env_partitions[:-1]]
test_partition = ToxicityDataset(env_partitions[-1][['id', 'toxicity', 'comment_text']], transform=t)[:]

print(train_envs[0]['x'].shape, test_partition['x'].shape)

In [ ]:
import pickle 
pickle.dump(train_envs, open('tenvs.pkl', 'wb'))

In [ ]:
import importlib
importlib.reload(ref)

args = {'lr': 0.001, \
         'n_iterations':70000, \
         'penalty_anneal_iters':1, \
         'l2_reg':1.0, \
         'pen_wgt':10, \
         'hid_layers':1, \
         'verbose':False}
base = ref.LinearInvariantRiskMinimization('cls')
model, errors, penalties, losses = base.train(train_envs, 1000, args)

In [ ]:
train_logits = base.predict(np.concatenate([train_envs[i]['x'] for i in range(len(train_envs))]), model)
train_labels = np.concatenate([train_envs[i]['y'] for i in range(len(train_envs))])
test_logits = base.predict(test_partition['x'], model)
test_labels = test_partition['y']

train_acc = ref.compute_loss(np.expand_dims(train_logits, axis=1), train_labels, ltype='ACC')
test_acc = ref.compute_loss(np.expand_dims(test_logits, axis=1), test_labels, ltype='ACC')
print('train score: {}'.format(train_acc))
print('test score: {}'.format(test_acc))


## Other

In [ ]:
import pickle
pickle.dump(model, open('none.pkl', 'wb'))

# Results

In [ ]:
resdir = '/Users/RobertAdragna/Documents/School/Fourth_Year/ESC499-Thesis/codebases/civil_liberties/results/0723_CMNIST/'

resdf = []
for f in os.listdir(resdir):
    if f.split('_')[0].isdigit() and f.split('_')[1] == 'baseline.pkl':  #assume every baseline has an irm 
        if (int(f.split('_')[0]) == 0) or ((int(f.split('_')[0]) % 2) == 0):
            base_data = pickle.load(open(join(resdir, f), 'rb'))
            entry = [base_data['id']['seed'], base_data['id']['env_splits'], base_data['id']['label_noise'], \
                     base_data['results']['train'], base_data['results']['test']]

            trt_data = pickle.load(open(join(resdir, '{}_{}'.format(f.split('_')[0], 'irm.pkl')), 'rb'))
            entry += [trt_data['results']['train'], trt_data['results']['test']]
            resdf.append(entry)

resdf = pd.DataFrame(resdf)
resdf.columns = ['seed', 'env_splits', 'label_noise', 'base_train', 'base_test', 'irm_train', 'irm_test']


test_mean = resdf.groupby(['env_splits', 'label_noise'])['base_train', 'base_test', 'irm_train', 'irm_test'].mean()
test_std = resdf.groupby(['env_splits', 'label_noise'])['base_train', 'base_test', 'irm_train', 'irm_test'].std()
assert (set(test_mean.columns) ^ set(test_mean.columns) == set()) and  (set(test_std.columns) ^ set(test_std.columns) == set())
test_full = test_mean.applymap(lambda x: "{0:01.3f}".format(x)) + test_std.applymap(lambda x: '  {}  '.format(u"\u00B1") +"{0:01.2f}".format(x))

test_full.head(30)

In [ ]:
resdf['id'] = np.ones(resdf.shape[0])
test_mean_agg = resdf.groupby(['id'])['base_train', 'base_test', 'irm_train', 'irm_test'].mean()
test_std_agg = resdf.groupby(['id'])['base_train', 'base_test', 'irm_train', 'irm_test'].std()
test_full_agg = test_mean_agg.applymap(lambda x: "{0:01.3f}".format(x)) + test_std_agg.applymap(lambda x: '  {}  '.format(u"\u00B1") +"{0:01.2f}".format(x))
test_full_agg.head()

## hyperparams

In [ ]:
#For Hyperparameters
resdir = '/Users/RobertAdragna/Documents/School/Fourth_Year/ESC499-Thesis/codebases/civil_liberties/results/hyperparam_tuning/0722_logreg/'

resdf = []
for f in os.listdir(resdir):
    if f.split('_')[0].isdigit() and f.split('_')[1] == 'baseline.pkl':  #assume every baseline has an irm 
        try:
            trt_data = pickle.load(open(join(resdir, '{}_{}'.format(f.split('_')[0], 'irm.pkl')), 'rb'))
            base_data = pickle.load(open(join(resdir, f), 'rb'))
        except:
            continue
        entry = [base_data['id']['seed'], base_data['id']['env_splits'], base_data['id']['label_noise']] + list(base_data['id']['algo_params'].values()) \
                 + [base_data['results']['train'], base_data['results']['test']]
        
        trt_data = pickle.load(open(join(resdir, '{}_{}'.format(f.split('_')[0], 'irm.pkl')), 'rb'))
        entry += [trt_data['results']['train'], trt_data['results']['test']]
        resdf.append(entry)

resdf = pd.DataFrame(resdf)
resdf.columns = ['seed', 'env_splits', 'label_noise'] \
                + ['lr', 'n_iterations', 'penalty_anneal_iters', 'l2_reg', 'pen_wgt', 'hid_layers', 'verbose'] \
                + ['base_train', 'base_test', 'irm_train', 'irm_test']


test = resdf.groupby(['lr', 'n_iterations', 'l2_reg', 'pen_wgt', 'hid_layers'])['base_train', 'base_test', 'irm_train', 'irm_test'].mean()
test.head(170)

In [ ]:
test = resdf.groupby(['l2_reg'])['base_train', 'base_test', 'irm_train', 'irm_test'].mean()
test.head()
#'lr', 'l2_reg', 'pen_wgt', 'hid_layers'

## save 

In [ ]:
test_full.to_latex(join(resdir, 'full_results_lgbtq.tex'), caption='LGBTQ sensitive attribute')
# test_full.to_excel(join(resdir, 'hyp_results.xls'))